In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/10623 Project")

In [ ]:
!pip install torchmetrics
!pip install torch-fidelity

!pip install pytorch-fid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.1 MB/s eta 0:00:00


In [ ]:
import os
import argparse
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from scipy import linalg
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.kid import KernelInceptionDistance
from datasets import load_dataset
import io

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_paths = []

        for filename in os.listdir(image_dir):
            self.image_paths.append(os.path.join(image_dir, filename))

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert("RGB")
        tensor_transform = transforms.ToTensor()
        return tensor_transform(image).to(torch.float)

    def __len__(self):
        return len(self.image_paths)


class HuggingFaceImageDataset(Dataset):
    def __init__(self, repo_id, split):
        self.repo_id = repo_id
        self.split = split
        self.images = []

        dataset = load_dataset(repo_id, split = split)

        for idx, item in enumerate(dataset):
            # image_bytes = item["image"]["bytes"]
            # image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

            image = item["image"].convert("RGB")
            self.images.append(image)


    def __getitem__(self, index):
        image = self.images[index]
        tensor_transform = transforms.ToTensor()
        return tensor_transform(image).to(torch.float)

    def __len__(self):
        return len(self.images)


def calculate_fid(real_dataset, fake_dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    fid = FrechetInceptionDistance(normalize = True).to(device)

    real_loader = DataLoader(real_dataset, shuffle = False, num_workers = 4)
    fake_loader = DataLoader(fake_dataset, shuffle = False, num_workers = 4)

    with torch.no_grad():
        for batch in real_loader:
            fid.update(batch.to(device), real = True)

    with torch.no_grad():
        for batch in fake_loader:
            fid.update(batch.to(device), real = False)

    return fid.compute()


def calculate_kid(real_dataset, fake_dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    kid = KernelInceptionDistance(subset_size = min(len(real_dataset), len(fake_dataset)), normalize = True).to(device)

    real_loader = DataLoader(real_dataset, shuffle = False, num_workers = 4)
    fake_loader = DataLoader(fake_dataset, shuffle = False, num_workers = 4)

    with torch.no_grad():
        for batch in real_loader:
            kid.update(batch.to(device), real = True)

    with torch.no_grad():
        for batch in fake_loader:
            kid.update(batch.to(device), real = False)

    return kid.compute()


def save_huggingface_dataset_to_dir(dataset, output_dir):
    os.makedirs(output_dir, exist_ok = True)
    for idx, image in enumerate(dataset.images):
        image_path = os.path.join(output_dir, f"{idx:05d}.png")
        image.save(image_path)
    return output_dir

if __name__ == "__main__":
    # repo_id = "Kariander1/img_sketch_100k"
    # repo_id_split = "test"

    # real_dir = "huggingface"
    # fake_dir = "outputs"

    # print("Calculating FID score between real and fake images.")
    # real_dataset = HuggingFaceImageDataset(repo_id, repo_id_split)
    # fake_dataset = ImageDataset(fake_dir)




    repo_id = "hengyiqun/10623-project-coco17"
    repo_id_split = "val[4500:]"

    # fake_dir = "sketch_depth_5_5"   # T2I-Adapter with adapter_conditioning_scale=[0.5, 0.5] (96.3514 - with 500 real images and 500 fake images) / (83.3015 - with 5000 real images and 500 fake images)
    # fake_dir = "sketch_depth"   # T2I-Adapter with adapter_conditioning_scale=[1.0, 1.0] (91.1659 - with 500 real images and 500 fake images) / (81.2796 - with 5000 real images and 500 fake images)
    # fake_dir = "GMU_Inference_30"   # Gated-Adapter (118.5316 - with 500 real images and 500 fake images) / (102.3450 - with 5000 real images and 500 fake images)
    fake_dir = "GMU_Inference_30_step10k"   # Gated-Adapter (94.4524 - with 500 real images and 500 fake images) / (81.9244 - with 5000 real images and 500 fake images)

    real_dataset = HuggingFaceImageDataset(repo_id, repo_id_split)
    fake_dataset = ImageDataset(fake_dir)

    print("Calculating FID score between real and fake images.")
    fid = calculate_fid(real_dataset, fake_dataset)
    print(fid)

    # print("Calculating KID score between real and fake images.")
    # kid = calculate_kid(real_dataset, fake_dataset)
    # print(kid)

Calculating FID score between real and fake images.
tensor(94.4524, device='cuda:0')


In [ ]:
######## For saving real_dataset into real_dir #######
######## Run this only if you want to use pytorch_fid, and do not have the real dataset downloaded yet #######
# real_dir = "huggingface_hengyiqun"
# save_huggingface_dataset_to_dir(real_dataset, real_dir)
#######################################################

'huggingface_hengyiqun'

In [ ]:
!python -m pytorch_fid --device cuda:0 huggingface_hengyiqun GMU_Inference_30

  0% 0/10 [00:02<?, ?it/s]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/pytorch_fid/__main__.py", line 3, in <module>
    pytorch_fid.fid_score.main()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_fid/fid_score.py", line 313, in main
    fid_value = calculate_fid_given_paths(args.path,
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_fid/fid_score.py", line 259, in calculate_fid_given_paths
    m1, s1 = compute_statistics_of_path(paths[0], model, batch_size,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_fid/fid_score.py", line 243, in compute_statistics_of_path
    m, s = calculate_activation_statistics(files, model, batch_size,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  